# Re-formatting plankton-specific marker genes fetched from different sources to create a custom database (DB) compatible with the powerful metagenomics-based taxonomic profiling tool [Motus](https://www.nature.com/articles/s41467-019-08844-4).  

See Motus' GitHub repo [here](https://github.com/motu-tool/mOTUs).

### NOTE: given that DB customization for Motus is not clearly explained in the Docs, we use a tool called [read_counter](https://github.com/AlessioMilanese/read_counter), which is a wrapper to run Motus using a customized DB.

Importantly, the default reference DB of marker genes used by Motus is not suitable for profiling marine planktonic ecosystems. Therefore we will create a plankton-specific marker gene DB to quantify relative abundance profiles across taxonomic groups. To achieve this, we will build on two well curated marker gene DBs: 

   - The huge catalog of phytoplankton psbO marker gene sequences, which encodes the manganese-stabilising polypeptide of the photosystem II oxygen evolving complex,  reported in this [paper](https://onlinelibrary.wiley.com/doi/epdf/10.1111/1755-0998.13592) and accessible [here](https://www.ebi.ac.uk/biostudies/studies/S-BSST761?query=A%20robust%20approach%20to%20estimate%20relative%20phytoplankton%20cell%20abundances%20from%20metagenomes). 

   - The [MZGdb](https://metazoogene.org/MZGdb) database and most specifically the "All Plankton Combo" files contain all data from the All Zooplankton and the All Ichthyoplankton combined files. This database was described in this [paper](https://link.springer.com/article/10.1007/s00227-021-03887-y). Here we will focus on DNA sequences for the barcode region of mitochondrial cytochrome oxidase I (COI).
   
#### The code developed in this Notebook is meant for developing our first proof-of-concept (POC1) biodiversity data asset, which focuses on the taxonomic composition found in a given environmental sample. In a nutshell, we assess relative abundances across numerous plankton taxonomic groups from metagenomics (MTG) datasets.  
 
</br>

Author: jay@newatlantis.io

In [ ]:
#pip install for outside requirements
!pip3 install -r requirements.txt

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import colorsys
from matplotlib.collections import PatchCollection

import Bio.SeqIO as bioseqio
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
#from Bio.Alphabet import IUPAC
from Bio import Entrez

from ete3 import NCBITaxa
from taxonomy_ranks import TaxonomyRanks

from subprocess import Popen, call, STDOUT, PIPE
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib
import json
import glob
import re
import gzip
import sys
import csv
import time
import io
import pathlib
from collections import OrderedDict

import pickle
import bz2 

from IPython.display import Image
from itertools import combinations
import itertools

In [ ]:
pd.set_option('mode.chained_assignment', None)

In [ ]:
matplotlib.rcParams['savefig.dpi'] = 1000
matplotlib.rcParams['figure.dpi'] = 1000
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context("paper")
sns.set(font='serif')
sns.set_style('ticks')

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
###Uncomment below if necessary
rendef = "png" #"pdf" 
fig_renderer = pio.renderers[rendef]
fig_renderer.width = 1000
fig_renderer.height = 1000
pio.renderers.default = rendef

Utility functions

In [ ]:
def format_tax_lbl(taxid = '94617'):
    """Function to properly format taxonomic labels compatible with MetaPhlan"""
    
    tax_lvls_lbls = ['species','genus','family', 'order', 'class', 'phylum', 'superkingdom'][::-1]
    rank_taxon = TaxonomyRanks(taxid)
    rank_taxon.get_lineage_taxids_and_taxanames()
    rank_dict =  list(rank_taxon.lineages.values())[0]
    tax_tree = list(rank_taxon.lineages.values())[0]
    
    #Parsing info on high rank tanks is optional, but can be quite handy when low ranks are unclassified: for higher rank assignment purposes if needed
    tax_ranks_list = [tax_tree[t][0].replace("NA","unclassified") for t in tax_lvls_lbls]
    ncbi_taxIDs_list = [str(tax_tree[t][1]) for t in tax_lvls_lbls]
    return ncbi_taxIDs_list,tax_ranks_list 

def parse_lineage(lineage_str):
    '''Function to correctly parse lineage using NCBI tax ID'''
    return (";".join(format_tax_lbl(lineage_str.split(';')[-1].replace('_', ' '))[-1])).replace(" ","_")

In [ ]:
# *Always* tell NCBI who you are
Entrez.email = "jayson.gutierrez@vliz.be"

In [ ]:
#Dumping our select marker gene set into this fasta
custom_db_fasta_fid = '../custom_db/CustomPhytoZooPlanktonMGs.fna'

### Processing/filtering COI (mitochondrial cytochrome oxidase I gene) sequences for Zooplankton sepecies

In [ ]:
coi_zooplankt_df = pd.read_csv("../data/MZGdata-coi__MZGdbALL__o00__A.csv", header=None)

A COI sequence from this highly curated gene sequence database looks like this 

In [ ]:
print(coi_zooplankt_df.iloc[1,30])

In [ ]:
print("This COI DB contains a total number of {} sequences".format(coi_zooplankt_df.shape[0]))

The following chunks of code will perform filtering and reformatting of the headers of each sequence in the original COI DB

In [ ]:
###Uncomment below to rebuild custom DB
filtered_coi_zooplankt_df = coi_zooplankt_df[coi_zooplankt_df.iloc[:,33].map(lambda s: isinstance(s, str))]
#Lets concentrate on organism name and corresponding COI sequence
min_coi_zooplankt_df = filtered_coi_zooplankt_df.iloc[:,[1,8,30]]
#Rename columns
#Use genebank accession to fetch a bunch of info needed to reformat headers
min_coi_zooplankt_df.columns = ["Species_name","Genebank_accession","Species_COI_seq"]
#Add full lineage
min_coi_zooplankt_df['Full_lineage'] = filtered_coi_zooplankt_df.iloc[:,33].map(lambda s: ";".join([l for l in s.split(';') if '_EXT' not in l and l!='']))
#Drop duplicate entries for genebank accessions
min_coi_zooplankt_df = min_coi_zooplankt_df.drop_duplicates(['Species_name'])
#Reset index
min_coi_zooplankt_df.reset_index(drop=True, inplace=True)

Iterating over each row in the filtered DF and creating a Seq object with formatted header, which is dumped into the dedicated file

In [ ]:
###Uncomment below to rebuild custom DB

# for i,seq_rec in min_coi_zooplankt_df.iterrows():

#     record = SeqRecord(
#     Seq(seq_rec['Species_COI_seq']),
#     id = seq_rec['Genebank_accession'] + '__' + seq_rec['Full_lineage'],
#     name="",
#     description="")
    
#     with open(custom_db_fasta_fid, "a") as output_handle:
#         bioseqio.write(record, output_handle, "fasta")       

### Processing/filtering psbO sequences for Phytoplankton sepecies.

Adding to fasta file already created above

In [ ]:
#Original psbO DB
psbO_db_fid = '../data/psbO_20210825.fna'

In [ ]:
print("An entry in the psbO DB (fasta format) looks as follows:\n")
!head $psbO_db_fid -n 10

In [ ]:
#Uncomment below to rebuild custom DB
with open(psbO_db_fid, "r") as handle:
    for (i,record) in enumerate(bioseqio.parse(handle, "fasta")):
        if(i>10):
            #Fetch standard header components and reformat full header
            seq_id, tax_lin = record.description.split(' ')[:2]
            record.description=''
            record.id = "{}__{}".format(seq_id, tax_lin)
            #Dump to fasta
            with open(custom_db_fasta_fid, "a") as output_handle:
                bioseqio.write(record, output_handle, "fasta") 

### Our final DB (concatenated gene markers for both Phyto and Zooplankton species)

In [ ]:
print("Total number of marker genes included in our customized DB of marker genes is:")
!grep -c '>' $custom_db_fasta_fid

### With the customized marker gene DB (for both zooplankton --COI sequences-- & phytoplankton --psbO sequences--) created above we can screen across metagenomic datasets in order to assess the taxonomic composition of the plankton community sequenced.

#### A short read in a MTG file looks like this:  

```@ERR1719507.4222 H2:D1NNJACXX:6:1101:5303:2333/1
AGCGAGCCCACTGTGTTCCCGGGGGACTGGGGGCCATTAGCGGCGTCAGACACGGGGGGGAGCGGGGTCTGACCATCCTGGGCCGGGACCCGGCCGTCCAGTTTGTCCAGCATGGCCCGGGCCGCCCCGTGCTTGGCCTGCTTCTTG
+
CCCFFFFFHHHHGGIIJJJJIJJJJGHIJJJJHDDDDDDDDDDCDBJJJHIGJJJJJJJJJJIJJJJJJJJIJJJJJJJJJJJJJJJEJJJJJJJJJJJJJDDDDDDDDDDDDDDDDDFFHJJIIJJJJJIJJJHGHHHFFFFFCCC```

### Analysis  
The following is a simple visualiztion illustrating the relative abundances across a bunch of plankton taxonomic groups (down to the species level) that we obtained from profiling a given TARA Ocean sample (ID: [ERR1719507](https://www.ebi.ac.uk/ena/browser/view/ERR1719507)) collected in the North Atlantic Ocean (offshore Cadiz, Spain, [Location 36.5533 N 6.5669 W](https://www.google.com/maps/place/36%C2%B033'11.9%22N+6%C2%B034'00.8%22W/@36.5534368,-6.5668384,17z/data=!4m5!3m4!1s0x0:0x9aa20881883fdb5f!8m2!3d36.5533!4d-6.5669)), on date/time=2009-09-15T18:00, using a PUMP (High Volume Peristaltic Pump). The sample material (particulate matter, including plankton (ENVO:xxxxxxxx)) was collected at a depth of 38-42 m, targeting a deep chlorophyll maximum layer (ENVO:xxxxxxxx) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.8-5 micrometres), and stored in liquid nitrogen for later detection of unicellular eukaryote (protist) nucleic acid sequences by pyrosequencing methods, and for later metagenomics/transcriptomics analysis. This sample has replicate sample(s): TARA_X000000407."

In [ ]:
with open('../results/ERR1719507_mapped_reads.map','r') as fid:
    lines = fid.readlines()
    
#Parsing lines
data = [re.split('\t|__',l.strip()) for l in lines if ';' in l]
#Pick only those with a given format to avoid noise
data = [l for l in data if len(l)==3]

In [ ]:
#Cast data into a DF
abund_df = pd.DataFrame(data, columns = ['GeneID','Lineage','Abundance'])
#Change data type
abund_df['Abundance'] = abund_df['Abundance'].astype(float)
#Sort data by Abundance
abund_df.sort_values('Abundance', ascending=False, inplace=True)
#Reset index for tractability purpose
abund_df.reset_index(drop=True, inplace=True)
#Reformatting lineage using NCBI taxID
abund_df['Lineage'] = abund_df['Lineage'].map(parse_lineage)
#Count total number of observations/hits across unique taxa
unique_taxa_abund_df = abund_df.groupby('Lineage')['Abundance'].sum().sort_values(ascending=False)
#Take log10 and make df
unique_taxa_log_abund_df = unique_taxa_abund_df.map(np.log10).reset_index()
#Cut off by a certain value
thresholded_df = unique_taxa_log_abund_df[unique_taxa_log_abund_df['Abundance']>=1]

Make a new DF with columns = taxonomic level, and then append at the end the abundance observed in the sample analyzed

In [ ]:
tax_enumeration_df_filtered = pd.DataFrame.from_records(thresholded_df['Lineage'].map(lambda s: s.split(';')).values)
#Name columns
tax_enumeration_df_filtered.columns = ['species','genus','family', 'order', 'class', 'phylum', 'superkingdom'][::-1]
#Add log-transformed abundance column 
tax_enumeration_df_filtered['log_abundance'] = thresholded_df['Abundance'].values

In [ ]:
#Peek at the new DF
tax_enumeration_df_filtered.head()

In [ ]:
fig = px.sunburst(tax_enumeration_df_filtered,#.query('superkingdom == "Eukaryota"'), 
                  path=['superkingdom','phylum', 'class', 'order', 'family', 'genus'], 
                  values='log_abundance', color='order')

fig.update_layout(
                title={
                    'text': "Species richness in TARA Ocean's sample (ID = ERR1719507)",
                    'y':0.985,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font_size':30,
                    'font_color':"black"})


# fig.update_yaxes(automargin=True)
# fig.update_xaxes(automargin=True)

fig.update_layout(
    autosize=False,
#     width=500,
#     height=500,
    margin=dict(
        l=1,
        r=1,
        b=4,
        t=50,
        pad=2
    ),
    paper_bgcolor="White",
)

fig.show(width=1000, height=1000)
# pio.write_image(fig, "CustomProkEukDB/SunburstTaxDist_DB_v2.3.png", width=1.5*1000, height=1*1000, scale =1.25)

### Observations  

- Zooplankton is clearly the numerically dominant taxonomic group in this sample.
- Based on the test performed above, one can conclude that the taxonomic profiling tools is quite effective at profiling the taxonomic composition of MTG datasets using our customized DB.  

- The tool is ready for large-scale testing (using more TARA Ocean MTG datasets) for better assessing the ability of the computational pipeline to characterize taxonomic diversity across samples collected across a great variety of oceanic provinces.  

- Once testing is achieved, we can confidently deploy the tool to characterize our future in-house collected MTG datasets.